In [1]:
from layup import routines
import numpy as np
import os

In [2]:
def AD_vec(vec):
    x, y, z = vec
    Ax = z
    Ay = 0.0
    Az = -x
    A = np.sqrt(Ax*Ax + Ay*Ay + Az*Az)
    Ax /= A
    Ay /= A
    Az /= A
    Dx = -x*y
    Dy = x*x + z*z
    Dz = -z*y
    D = np.sqrt(Dx*Dx + Dy*Dy + Dz*Dz)
    Dx /= D
    Dy /= D
    Dz /= D
    return ([Ax, Ay, Az], [Dx, Dy, Dz])

In [3]:
obs_list = []
with open('/Users/mholman/03666_out.txt') as file:
    for i, line in enumerate(file):
        objID, obsCode, mag, filt, jd_tdb, rho_x, rho_y, rho_z, xe, ye, ze, ast_unc = line.rstrip().split()
        jd_tdb = float(jd_tdb)
        rho_x, rho_y, rho_z = float(rho_x), float(rho_y), float(rho_z)
        xe, ye, ze = float(xe), float(ye), float(ze)
        ast_unc = float(ast_unc)/206265
        Av, Dv = AD_vec((rho_x, rho_y, rho_z))
        rho = [rho_x, rho_y, rho_z]
        re = [xe, ye, ze]
        ve = [0., 0., 0.]
        obs = routines.Observation(jd_tdb, re, ve, rho, Av, Dv, ast_unc, ast_unc)
        obs_list.append(obs)


        

In [4]:
GMtotal = 0.0002963092748799319
AU_M = 149597870700
SPEED_OF_LIGHT = 2.99792458e8 * 86400.0 / AU_M

In [5]:
solns = routines.gauss(GMtotal, obs_list[4290], obs_list[4308], obs_list[4311], 0.0001, SPEED_OF_LIGHT)

In [6]:
solns[0].root

3.347895837553503

In [7]:
assist_ephem = routines.get_ephem("/Users/mholman/Library/Caches/layup/")

In [8]:
x = routines.run_from_vector(assist_ephem, obs_list)

gauss indices: 4300 4308 4311
chi2: 28305.4
lambda: 2.12726e+07
dX
-0.000400072
 0.000821698
 0.000340927
-0.000109324
 0.000322614
 0.000136549
chi2: 2710.78
lambda: 1.06363e+07
dX
-0.000640446
  0.00164414
  0.00072191
 1.18867e-07
-2.18375e-05
-9.27258e-06
chi2: 2553.33
lambda: 5.31816e+06
dX
 -0.00120814
  0.00310366
  0.00136266
 5.49218e-07
-4.19186e-05
 -1.7791e-05
chi2: 2269.09
lambda: 2.65908e+06
dX
 -0.00215771
  0.00554305
  0.00243369
 9.67324e-07
-7.48324e-05
-3.17597e-05
chi2: 1801.84
lambda: 1.32954e+06
dX
 -0.00347457
  0.00892596
  0.00391897
 1.52207e-06
-0.000120419
-5.11058e-05
chi2: 1159.8
lambda: 664770
dX
 -0.00465792
   0.0119659
  0.00525366
 1.96079e-06
-0.000161244
-6.84286e-05
chi2: 517.27
lambda: 332385
dX
 -0.00464196
   0.0119248
  0.00523566
  1.8422e-06
-0.000160429
-6.80787e-05
chi2: 132.113
lambda: 166192
dX
 -0.00302761
  0.00777762
  0.00341484
 1.12662e-06
-0.000104463
-4.43262e-05
chi2: 21.6149
lambda: 83096.2
dX
 -0.00116153
  0.00298384
  0.0013

In [9]:
x.ndof, x.niter, x.state, x.csq

(8618,
 3,
 [1.985611682748701,
  -2.479380117033196,
  -1.0907277689636068,
  0.007773290630228675,
  0.004329331766536715,
  0.0014714188352594674],
 1838.4411773203663)